# Image classification

影像分類（image classification）是將影像賦予標籤或類別的過程。不同於文本或音頻分類，輸入是影像上的像素值。影像分類有很多應用，例如在自然災害後檢測損壞情況，監測農作物的健康狀況或幫助篩檢醫學影像中的病徵。

以下內容會說明：
1. 在 Food-101 資料集上微調 ViT 模型，以便將影像中的食物進行分類。
2. 使用微調後的模型進行推論。

>以下所演示的任務得以使用的模型架構是：[BEiT](https://huggingface.co/docs/transformers/model_doc/beit), [BiT](https://huggingface.co/docs/transformers/model_doc/bit), [ConvNeXT](https://huggingface.co/docs/transformers/model_doc/convnext), [ConvNeXTV2](https://huggingface.co/docs/transformers/model_doc/convnextv2), [CvT](https://huggingface.co/docs/transformers/model_doc/cvt), [Data2VecVision](https://huggingface.co/docs/transformers/model_doc/data2vec-vision), [DeiT](https://huggingface.co/docs/transformers/model_doc/deit), [DiNAT](https://huggingface.co/docs/transformers/model_doc/dinat), [EfficientFormer](https://huggingface.co/docs/transformers/model_doc/efficientformer), [EfficientNet](https://huggingface.co/docs/transformers/model_doc/efficientnet), [ImageGPT](https://huggingface.co/docs/transformers/model_doc/imagegpt), [LeViT](https://huggingface.co/docs/transformers/model_doc/levit), [MobileNetV1](https://huggingface.co/docs/transformers/model_doc/mobilenet_v1), [MobileNetV2](https://huggingface.co/docs/transformers/model_doc/mobilenet_v2), [MobileViT](https://huggingface.co/docs/transformers/model_doc/mobilevit), [NAT](https://huggingface.co/docs/transformers/model_doc/nat), [Perceiver](https://huggingface.co/docs/transformers/model_doc/perceiver), [PoolFormer](https://huggingface.co/docs/transformers/model_doc/poolformer), [RegNet](https://huggingface.co/docs/transformers/model_doc/regnet), [ResNet](https://huggingface.co/docs/transformers/model_doc/resnet), [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer), [Swin Transformer](https://huggingface.co/docs/transformers/model_doc/swin), [Swin Transformer V2](https://huggingface.co/docs/transformers/model_doc/swinv2), [VAN](https://huggingface.co/docs/transformers/model_doc/van), [ViT](https://huggingface.co/docs/transformers/model_doc/vit), [ViT Hybrid](https://huggingface.co/docs/transformers/model_doc/vit_hybrid), [ViTMSN](https://huggingface.co/docs/transformers/model_doc/vit_msn)

在開始之前，確認已安裝所有必要的套件：

Before you begin, make sure you have all the necessary libraries installed:

In [ ]:
!pip install -q transformers==4.30.0 datasets evaluate

## Load Food-101 dataset


首先，從 Hugging Face 資料庫中載入 Food-101 資料集的部分資料。這能預先進行多項實驗，確保在完整資料集上進行更多的訓練前，所有步驟都能夠正常運作。

In [ ]:
from datasets import load_dataset

food = load_dataset("/home/jovyan/ta-shared-ii/datas/new_food-10/", split="train")

In [ ]:
food

In [ ]:
item = next(iter(food))

In [ ]:
print(item['label'])
item['image']

使用 train_test_split 方法將資料集的訓練部分再進一步切分成訓練集和測試集：

In [ ]:
food = food.train_test_split(test_size=0.2)

In [ ]:
food

查看其中一個樣本：

In [ ]:
food["train"][0]

資料集中每個範例都有兩個欄位：
* image: 包含食物的 PIL(pillow 格式) 影像
* label: 食物的標籤類別

為了讓模型更容易從標籤 id 中讀取名稱，創建一個將標籤名稱對應到整數以及反對應的字典：

In [ ]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
labels[:5]

現在，按照標籤 id 轉換成名稱：

In [ ]:
id2label[str(3)]

---

## Preprocess


接下來的步驟是載入 ViT 模型使用的影像處理器，將影像處理成張量：

In [ ]:
from transformers import AutoImageProcessor
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
image_processor

---
將影像進行轉換，使模型更具一般性以應付過擬合的情況。這裡會使用的 torchvision 中 transforms 的模組，但也能替換成其他適用的影像處理套件。

隨機裁減影像的一部份，將其調整影像大小，並使用影像的平均值和標準差進行標準化：

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

接下來創建一個預處理函數，轉換並回傳影像的像素值作為模型的輸入：

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

要在整個資料集上應用預處理函數，可以使用 Hugging Face 資料集的 [with_transform](https://huggingface.co/docs/datasets/v2.11.0/en/package_reference/main_classes#datasets.Dataset.with_transform) 方法。當載入資料集的一個元素時，轉換會即時套用：

In [ ]:
food = food.with_transform(transforms)

In [ ]:
food

In [ ]:
import matplotlib.pyplot as plt
import torch

In [ ]:
item = next(iter(food['train']))

In [ ]:
print(item['label'])
print(item['pixel_values'].size())
plt.imshow(torch.permute(item['pixel_values'], (1, 2, 0)))

現在使用 [DefaultDataCollator](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/data_collator#transformers.DefaultDataCollator) 創建一個批次樣本。與 Hugging face 裡 Transformers 的其他資料收集器不同，DefaultDataCollator 不會套用額外的預處理，例如填充（padding）。

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Evaluate

在訓練過程中加入評估指標通常有助於評估模型的表現。可以使用 Hugging Face 的 [Evaluate](https://huggingface.co/docs/evaluate/index) 函式庫快速載入評估方法。在此任務上載入 [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) 指標（請參閱 Hugging Face 的 Evaluate [快速導覽](https://huggingface.co/docs/evaluate/a_quick_tour)，以了解如何載入和計算指標的詳細資訊）：

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

然後創建一個函數，將預測及標籤使用 [compute](https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/main_classes#evaluate.EvaluationModule.compute) 以計算準確度：

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

定義完 compute_metrics 函數，在訓練設定時會再次使用到它。

>如果不熟悉使用 [Trainer]((https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer)) 微調模型，請參考此[基本教程](https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer)！

現在已準備好開始訓練模型了！使用 [AutoModelForImageClassification](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoModelForImageClassification) 載入 ViT。指定標籤的數量以及標籤的對應方式：

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

接著的階段，只剩以下三個步驟：

1. 在 [TrainingArguments](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments) 中定義訓練的超參數。請務必留意資料集中未使用的資訊，設定 remove_unused_columns=False 可以防止被刪除未使用到的資訊！例如 image，這會導致無法獲得 pixel_values。另一個必需設定的參數是 output_dir，指定模型儲存的位置。通過設定 push_to_hub=True 將模型上傳至 Hub（需要登入 Hugging Face 才能上傳模型）。在每個 epoch 結束時，[Trainer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer) 將評估準確性並儲存訓練模型。
2. 將訓練參數、模型、資料集、預處理器、資料收集器以及計算評估指標函數傳遞給 [Trainer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer)。
3. 呼叫 [train](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer.train) 來微調模型。

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
#     logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [ ]:
from transformers.integrations import MLflowCallback
trainer.remove_callback(MLflowCallback)
trainer.train()

## Inference

現在，微調後的模型以存放在指定路徑，並可使用它來進行推論！

載入想要進行推論的影像：

In [ ]:
ds = load_dataset("/home/jovyan/ta-shared-ii/datas/new_food-10/", split="validation")
image = ds["image"][0]

In [ ]:
image

使用微調後的模型進行推論最簡單的方法是在 pipline() 中設定。藉由指定的模型建構一個影像分類的 pipeline，然後將影像傳遞給它：

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_awesome_food_model/checkpoint-141/")
classifier(image)

載入影像處理器對影像進行預處理，並以 PyTorch 的張量型態回傳作為輸入：

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("my_awesome_food_model/checkpoint-141/")
inputs = image_processor(image, return_tensors="pt")

將輸入傳遞給模型，並回傳 logits（尚未經過 softmax）：

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("my_awesome_food_model/checkpoint-141/")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

---
## Conclusion - HuggingFace Workflows

![](https://hackmd.io/_uploads/HkEzgZdwh.png)